In [1]:
from pymystem3 import Mystem
import nltk
import pandas as pd

import re
import json
from os import listdir
from os import path

In [2]:
articles_dir = "parsed_articles"
article_files = [f for f in listdir(articles_dir) if path.isfile(path.join(articles_dir, f))]

articles = dict()

for article_file in article_files:
    article_path = path.join(articles_dir, article_file)
    # print(article_path)
    with open(article_path, 'r', encoding="UTF-8") as f:
        article = f.read()
        article_json = json.loads(article)
        articles[article_json["article_id"]] = article_json
        # print(article_json)

In [3]:
from nltk.corpus import stopwords
m = Mystem()
stopwords = set(stopwords.words('russian'))

def prepare_text(text):
    prepared_text = text
    prepared_text = prepared_text.replace("\n", " ")
    prepared_text = re.sub(r'[^\w ]', '', prepared_text)
    prepared_text = prepared_text.lower()
    prepared_text = " ".join(filter(lambda x: x not in stopwords, prepared_text.split(" ")))
    prepared_text = ''.join(m.lemmatize(prepared_text))
    prepared_text = prepared_text.strip()
    
    return prepared_text

def prepare_tags(tags):
    prepared_tags = tags
    prepared_tags = map(lambda tag: tag.lower(), prepared_tags)
    prepared_tags = map(lambda tag: tag.replace("-", " "), prepared_tags)
    
    return list(prepared_tags)

In [4]:
df = pd.DataFrame(articles.values())
df['content'] = df['content'].apply(prepare_text)
df['tags'] = df['tags'].apply(prepare_tags)
df

,article_id,article_name,content,tags
0,729250,Rust Foundation хочет запретить менять цвета л...,предупреждение низко это запаздывать первоапре...,"[rust, копирайт]"
1,729060,Самый детальный разбор закона об электронных п...,новый закон электронный повестка потенциально ...,"[законодательство, мобилизация, повестка, госу..."
2,728856,Идеальное компьютерное кресло — миф или реальн...,привет хабр продолжать разбираться существоват...,"[рабочее место, рабочее место программиста, кр..."
3,728644,Стилмэнинг и идеологический тест Тьюринга,мысль начинаться наталкиваться препятствие лев...,"[стилменинг, идеологический тест тьюринга, сол..."
4,728748,RMT в World of Warcraft: почему Blizzard терпи...,промпт game world of warcraft fantasy doge mon...,"[world of warcraft, blizzard, kandinsky art, н..."
...,...,...,...,...
994,726292,Как работает веб-браузер (с картинками),иллюстрация предоставлять growtika браузер ст...,"[браузеры, dns поиск, tcp, tls, токенизация, j..."
995,726306,Ближайшие бесплатные мероприятия по разработке...,1 апрель начало 1100 мск суббота онлайн t...,"[митап, конференция, mobile, документация, rab..."
996,726316,Bash скрипты,работа командный строка linux shell скрипт на...,"[bash, bash скрипт, bash scripting, shell]"
997,726308,ADD – добавим гибкости,такой add короче add английский аббревиатура...,"[гибкая разработка по, методология разработки,..."


In [25]:
# list(filter(lambda t: len(t)<3, df["tags"]))

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['tags'])
len(mlb.classes_)

3648

In [6]:
df["tags_bins"] = df["tags"].apply(lambda x: mlb.transform([x])[0])
df

,article_id,article_name,content,tags,tags_bins
0,729250,Rust Foundation хочет запретить менять цвета л...,предупреждение низко это запаздывать первоапре...,"[rust, копирайт]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,729060,Самый детальный разбор закона об электронных п...,новый закон электронный повестка потенциально ...,"[законодательство, мобилизация, повестка, госу...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,728856,Идеальное компьютерное кресло — миф или реальн...,привет хабр продолжать разбираться существоват...,"[рабочее место, рабочее место программиста, кр...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,728644,Стилмэнинг и идеологический тест Тьюринга,мысль начинаться наталкиваться препятствие лев...,"[стилменинг, идеологический тест тьюринга, сол...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,728748,RMT в World of Warcraft: почему Blizzard терпи...,промпт game world of warcraft fantasy doge mon...,"[world of warcraft, blizzard, kandinsky art, н...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
994,726292,Как работает веб-браузер (с картинками),иллюстрация предоставлять growtika браузер ст...,"[браузеры, dns поиск, tcp, tls, токенизация, j...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
995,726306,Ближайшие бесплатные мероприятия по разработке...,1 апрель начало 1100 мск суббота онлайн t...,"[митап, конференция, mobile, документация, rab...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
996,726316,Bash скрипты,работа командный строка linux shell скрипт на...,"[bash, bash скрипт, bash scripting, shell]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
997,726308,ADD – добавим гибкости,такой add короче add английский аббревиатура...,"[гибкая разработка по, методология разработки,...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
# mlb.inverse_transform(y)[0]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size = 0.3, random_state=120)

In [80]:
# train, test = train_test_split(df, test_size = 0.25, random_state=95)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tdidf_vectorizer = TfidfVectorizer(decode_error='ignore')
X_train_tdidf_vectorized = tdidf_vectorizer.fit_transform(X_train['content'])
X_test_tdidf_vectorized = tdidf_vectorizer.transform(X_test['content'])

In [29]:
y_test[0]

array([0, 0, 0, ..., 0, 0, 0])

In [28]:
# classifier v1
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier

#обучаем подель SVM

import warnings
warnings.filterwarnings('ignore')
print("warnings will be ignored")

model = OneVsRestClassifier(LinearSVC(random_state = 50))
model.fit(X_train_tdidf_vectorized, y_train)

warnings will be ignored


OneVsRestClassifier(estimator=LinearSVC(random_state=50))

In [10]:
# predictions = model.predict(X_test_tdidf_vectorized)

# pred_tags = mlb.inverse_transform(predictions)
# list(zip(X_test['article_id'], pred_tags))
# list(map(lambda a: (f"https://habr.com/ru/articles/{a[0]}", a[1]), zip(X_test['article_id'], pred_tags)))
# [print(f"tags: {tags} \nclassified: {pred_tags}\n\n") for tags, pred_tags in zip(X_test['tags'], pred_tags)]

In [ ]:
# may be useful

# LogReg_pipeline = Pipeline([
#                 ('tfidf', TfidfVectorizer()),
#                 ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
#             ])
# for category in categories:
#     print('... Processing {}'.format(category))
#     # train the model using X_dtm & y
#     LogReg_pipeline.fit(X_train, train[category])
#     # compute the testing accuracy
#     prediction = LogReg_pipeline.predict(X_test)
#     print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

In [30]:
# var 2
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier

count_vectorizer = CountVectorizer(decode_error='ignore')
X_train_count_vectorized = count_vectorizer.fit_transform(X_train['content'])
X_test_count_vectorized = count_vectorizer.transform(X_test['content'])

#обучаем подель SVM

import warnings
warnings.filterwarnings('ignore')
print("warnings will be ignored")

model = OneVsRestClassifier(LinearSVC(random_state = 40))
model.fit(X_train_count_vectorized, y_train)

warnings will be ignored


OneVsRestClassifier(estimator=LinearSVC(random_state=40))

In [9]:
predictions = model.predict(X_test_count_vectorized)
pred_tags = mlb.inverse_transform(predictions)
# list(zip(X_test['article_id'], pred_tags))
for tags, pred_tags in zip(X_test['tags'], pred_tags):
    print(f"tags: {tags} \nclassified: {pred_tags}\n\n")

NameError: name 'model' is not defined

In [33]:
model.score(X_test_count_vectorized, y_test)

0.008

In [39]:
# classifier var 3
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

# LogReg_pipeline = Pipeline([
#                 ('tfidf', TfidfVectorizer()),
#                 ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
#             ])
# solvers = ['liblinear', 'newton-cg', 'sag', 'saga', 'lbfgs']

model3 = OneVsRestClassifier(LogisticRegression(solver='lbfgs', random_state = 50))

# print('Processing...')
# model3.fit(X_train_tdidf_vectorized, y_train)

In [25]:
model3

OneVsRestClassifier(estimator=LogisticRegression(random_state=50))

In [40]:
# predictions = model3.predict(X_test_tdidf_vectorized)

# pred_tags = mlb.inverse_transform(predictions)
# list(zip(X_test['article_id'], pred_tags))
# list(map(lambda a: (f"https://habr.com/ru/articles/{a[0]}", a[1]), zip(X_test['article_id'], pred_tags)))
# [print(f"tags: {tags} \nclassified: {pred_tags}\n\n") for tags, pred_tags in zip(X_test['tags'], pred_tags)]

In [ ]:
# var 4
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier

count_vectorizer = CountVectorizer(decode_error='ignore')
X_train_count_vectorized = count_vectorizer.fit_transform(X_train['content'])
X_test_count_vectorized = count_vectorizer.transform(X_test['content'])


import warnings
warnings.filterwarnings('ignore')
print("warnings will be ignored")

model4 = OneVsRestClassifier(MultinomialNB(force_alpha=False))
model4.fit(X_train_count_vectorized, y_train)

warnings will be ignored


In [20]:
import pickle

with open("logistic_regression_seed_120.model", 'wb') as f:
    pickle.dump(model3, f)